## Step 3: Union events across towers |  Load Events!



In [1]:
import pandas as pd
from typing import Tuple, Optional

In [2]:
# Read the CSV file
df = pd.read_csv("fully_labeled_weather_data_with_events.csv")

# 2️⃣ Move 'tower' and 'timestamp' to the first two columns
cols = df.columns.tolist()
priority_cols = ['tower', 'timestamp']
remaining_cols = [c for c in cols if c not in priority_cols]
df = df[priority_cols + remaining_cols]

# Drop columns ending with '_min' or '_meets_duration'
df = df.loc[:, ~df.columns.str.endswith(('_min', '_meets_duration'))]

#remove these columns
df = df.drop(columns=['event_count','active_events','event_durations', 'has_any_event'])


pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Check the first few rows
df.head(20)


,tower,timestamp,TempC_015m,TempC_030m,RelHum_015m,AbsHum_015m,WSpdMph_015m,WSpdMph_030m,PkWSpdMph_015m,PkWSpdMph_030m,VSSpdMph_015m,VSSpdMph_030m,BarPresMb_015m,Sigma_015m,Sigma_030m,SigPhi_015m,SigPhi_030m,WDir_015m,WDir_030m,PrecipIn_015m,event_E3_LowTemp_lt0,event_E4_HighWind_Peak_gt25,event_E5_LowWind_lt2,event_E6_HighTemp_gt24,SolarRadWm2_015m,TempC_002m,TempC_035m,TempC_060m,RelHum_002m,AbsHum_002m,WSpdMph_035m,WSpdMph_060m,PkWSpdMph_035m,PkWSpdMph_060m,VSSpdMph_060m,SolarRadWm2_002m,BarPresMb_002m,Sigma_035m,Sigma_060m,SigPhi_060m,WDir_035m,WDir_060m,PrecipIn_002m,TempC_010m,RelHum_010m,AbsHum_010m,WSpdMph_010m,PkWSpdMph_010m,VSSpdMph_010m,SolarRadWm2_010m,BarPresMb_010m,Sigma_010m,SigPhi_010m,WDir_010m,PrecipIn_010m,TempC_025m,WSpdMph_025m,PkWSpdMph_025m,VSSpdMph_025m,Sigma_025m,SigPhi_025m,WDir_025m,TempC_033m,WSpdMph_033m,PkWSpdMph_033m,VSSpdMph_033m,Sigma_033m,SigPhi_033m,WDir_033m
0,TOWA,2017-01-01 05:00:00+00:00,3.600000,3.6,97.4,6.0,2.2,4.4,5.9,10.2,0.00000,-0.100000,985.30000,26.5,15.5,18.9,12.6,268.0,274.0,0.015748,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TOWB,2017-01-01 05:00:00+00:00,3.900000,3.8,99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,986.40000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TOWD,2017-01-01 05:00:00+00:00,3.900000,NaN,97.2,6.1,2.2,NaN,5.5,NaN,-0.04474,NaN,986.40000,26.3,NaN,12.6,NaN,257.6,NaN,NaN,False,False,False,False,NaN,3.9,3.8,3.7,96.7,6.1,3.0,4.6,7.3,7.8,-0.15659,-2.4,986.4,18.0,14.2,9.4,272.7,275.3,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TOWF,2017-01-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TOWS,2017-01-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.366668,9.6,15.3,0.542,13.9,8.3,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TOWY,2017-01-01 05:00:00+00:00,4.166666,NaN,95.7,NaN,7.6,NaN,13.1,NaN,-0.59000,NaN,982.92926,17.5,NaN,9.0,NaN,274.0,NaN,0.010000,False,False,False,False,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.777777,8.8,15.3,-0.182,16.7,11.2,274.0
6,TOWA,2017-01-01 05:15:00+00:00,3.600000,3.6,97.3,6.0,2.4,4.7,6.7,7.7,0.00000,-0.100000,985.30000,19.3,13.9,14.3,9.8,252.0,264.0,0.011811,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TOWB,2017-01-01 05:15:00+00:00,3.800000,3.7,99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,986.40000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TOWD,2017-01-01 05:15:00+00:00,3.900000,NaN,97.5,6.1,2.1,NaN,6.2,NaN,-0.06711,NaN,986.40000,32.7,NaN,14.2,NaN,233.2,NaN,NaN,False,False,False,False,NaN,3.9,3.8,3.7,96.7,6.1,3.3,5.0,7.1,8.7,-0.02237,-1.3,986.4,21.2,16.2,10.6,258.4,266.4,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TOWF,2017-01-01 05:15:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### EVENT SUMMARY:

In [3]:



import pandas as pd

# List all boolean event columns
event_cols = [
    'event_E3_LowTemp_lt0',
    'event_E4_HighWind_Peak_gt25',
    'event_E5_LowWind_lt2',
    'event_E6_HighTemp_gt24'
]

# Ensure the columns are boolean
df[event_cols] = df[event_cols].astype(bool)

# Count how many events are True per row
df["true_event_count"] = df[event_cols].sum(axis=1)

# Filter: rows where more than 2 events are True simultaneously (per tower)
multiple_events_df = df[df["true_event_count"] > 2]

# Optional: group by tower if you want to see which towers are affected
tower_event_summary = multiple_events_df.groupby("tower")["true_event_count"].count().reset_index()

print("✅ Rows with more than two active events:")
print(multiple_events_df[["tower", "timestamp", "true_event_count"]])

print("\n📊 Summary of towers with >2 events:")
print(tower_event_summary)


✅ Rows with more than two active events:
        tower                  timestamp  true_event_count
646309   TOWB  2020-01-28 12:30:00+00:00                 3
1074314  TOWD  2022-02-09 18:00:00+00:00                 3

📊 Summary of towers with >2 events:
  tower  true_event_count
0  TOWB                 1
1  TOWD                 1


In [4]:
# Check all unique tower types
print("Unique tower types:", df['tower'].unique())

# Split into separate DataFrames per tower
tower_dfs = {tower: df[df['tower'] == tower].copy() for tower in df['tower'].unique()}

# Example: Access one tower's dataset
print("\nExample – TOWA DataFrame:")
print(tower_dfs['TOWA'].head())

Unique tower types: ['TOWA' 'TOWB' 'TOWD' 'TOWF' 'TOWS' 'TOWY']

Example – TOWA DataFrame:
   tower                  timestamp  TempC_015m  TempC_030m  RelHum_015m  \
0   TOWA  2017-01-01 05:00:00+00:00         3.6         3.6         97.4   
6   TOWA  2017-01-01 05:15:00+00:00         3.6         3.6         97.3   
12  TOWA  2017-01-01 05:30:00+00:00         3.6         3.7         97.4   
18  TOWA  2017-01-01 05:45:00+00:00         3.7         3.7         97.4   
24  TOWA  2017-01-01 06:00:00+00:00         3.8         3.8         97.3   

    AbsHum_015m  WSpdMph_015m  WSpdMph_030m  PkWSpdMph_015m  PkWSpdMph_030m  \
0           6.0           2.2           4.4             5.9            10.2   
6           6.0           2.4           4.7             6.7             7.7   
12          6.0           2.8           4.3             6.0             6.8   
18          6.0           2.5           4.2             5.1             6.8   
24          6.0           2.2           4.0             4

In [5]:
import pandas as pd
import numpy as np

# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Compute % of nulls per column and tower
null_percentage = df.groupby('tower').apply(lambda x: x.isnull().mean() * 100).round(2)

# Define non-overlapping ranges: 0–10%, 10–20%, ..., 90–100%
bins = np.arange(0, 110, 10)  # 0,10,20,...,100
range_labels = [f"{bins[i]}–{bins[i+1]}% nulls" for i in range(len(bins)-1)]

# Store results in dictionary
ranges = {label: [] for label in range_labels}

for tower in null_percentage.index:
    tower_data = null_percentage.loc[tower]
    for i in range(len(bins)-1):
        low, high = bins[i], bins[i+1]
        cols = list(tower_data[(tower_data >= low) & (tower_data < high)].index)
        ranges[range_labels[i]].append((tower, cols))

# Print summary
for category, values in ranges.items():
    print(f"\n===== {category} =====")
    for tower, cols in values:
        if len(cols) > 0:
            print(f"Tower {tower}: {len(cols)} columns → {cols}")



===== 0–10% nulls =====
Tower TOWA: 24 columns → ['tower', 'timestamp', 'TempC_015m', 'TempC_030m', 'RelHum_015m', 'AbsHum_015m', 'WSpdMph_015m', 'PkWSpdMph_015m', 'PkWSpdMph_030m', 'VSSpdMph_015m', 'VSSpdMph_030m', 'BarPresMb_015m', 'Sigma_015m', 'Sigma_030m', 'SigPhi_015m', 'SigPhi_030m', 'WDir_015m', 'WDir_030m', 'PrecipIn_015m', 'event_E3_LowTemp_lt0', 'event_E4_HighWind_Peak_gt25', 'event_E5_LowWind_lt2', 'event_E6_HighTemp_gt24', 'true_event_count']
Tower TOWB: 8 columns → ['tower', 'timestamp', 'BarPresMb_015m', 'event_E3_LowTemp_lt0', 'event_E4_HighWind_Peak_gt25', 'event_E5_LowWind_lt2', 'event_E6_HighTemp_gt24', 'true_event_count']
Tower TOWD: 30 columns → ['tower', 'timestamp', 'TempC_015m', 'RelHum_015m', 'AbsHum_015m', 'BarPresMb_015m', 'Sigma_015m', 'SigPhi_015m', 'event_E3_LowTemp_lt0', 'event_E4_HighWind_Peak_gt25', 'event_E5_LowWind_lt2', 'event_E6_HighTemp_gt24', 'TempC_002m', 'TempC_035m', 'TempC_060m', 'RelHum_002m', 'AbsHum_002m', 'WSpdMph_035m', 'WSpdMph_060m', '

/tmp/ipykernel_3279310/3669835264.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  null_percentage = df.groupby('tower').apply(lambda x: x.isnull().mean() * 100).round(2)


In [6]:
def filter_dfs_below_threshold(df, ranges, threshold=100):
    """
    Filter all towers, keeping columns whose null % is below a threshold.
    Always keeps 'tower', 'timestamp' and 'true_event_count'.
    """
    filtered_dfs = {}
    
    # Find all ranges below threshold
    selected_ranges = [r for r in ranges.keys() if int(r.split("–")[1].replace("% nulls","")) <= threshold]
    
    for tower in df['tower'].unique():
        cols_to_keep = ['tower', 'timestamp']  # <-- keep true_event_count
        
        # Collect columns from all selected ranges
        for r in selected_ranges:
            for t, cols in ranges[r]:
                if t == tower:
                    cols_to_keep.extend(cols)
        
        # Remove duplicates (some columns may appear in multiple ranges)
        cols_to_keep = list(dict.fromkeys(cols_to_keep))
        
        # Guard: keep only columns that actually exist in the dataframe
        cols_to_keep = [c for c in cols_to_keep if c in df.columns]
        
        filtered_dfs[tower] = df[df['tower'] == tower][cols_to_keep].copy()
        filtered_dfs[tower] = filtered_dfs[tower].drop("true_event_count", axis=1)
    
    return filtered_dfs

filtered_dfs = filter_dfs_below_threshold(df, ranges, threshold=100)


In [7]:
{tower: list(df.columns) for tower, df in filtered_dfs.items()}

{'TOWA': ['tower',
  'timestamp',
  'TempC_015m',
  'TempC_030m',
  'RelHum_015m',
  'AbsHum_015m',
  'WSpdMph_015m',
  'PkWSpdMph_015m',
  'PkWSpdMph_030m',
  'VSSpdMph_015m',
  'VSSpdMph_030m',
  'BarPresMb_015m',
  'Sigma_015m',
  'Sigma_030m',
  'SigPhi_015m',
  'SigPhi_030m',
  'WDir_015m',
  'WDir_030m',
  'PrecipIn_015m',
  'event_E3_LowTemp_lt0',
  'event_E4_HighWind_Peak_gt25',
  'event_E5_LowWind_lt2',
  'event_E6_HighTemp_gt24',
  'WSpdMph_030m'],
 'TOWB': ['tower',
  'timestamp',
  'BarPresMb_015m',
  'event_E3_LowTemp_lt0',
  'event_E4_HighWind_Peak_gt25',
  'event_E5_LowWind_lt2',
  'event_E6_HighTemp_gt24',
  'TempC_015m',
  'TempC_030m',
  'WSpdMph_015m',
  'WSpdMph_030m',
  'PkWSpdMph_015m',
  'PkWSpdMph_030m',
  'Sigma_015m',
  'Sigma_030m',
  'WDir_015m',
  'WDir_030m',
  'PrecipIn_015m',
  'VSSpdMph_015m',
  'VSSpdMph_030m',
  'SigPhi_015m',
  'SigPhi_030m',
  'RelHum_015m'],
 'TOWD': ['tower',
  'timestamp',
  'TempC_015m',
  'RelHum_015m',
  'AbsHum_015m',
  'BarP

In [ ]:
# Temporarily disable truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # No width limit
pd.set_option('display.max_colwidth', None) # Full column names


In [9]:
filtered_dfs['TOWA']

,tower,timestamp,TempC_015m,TempC_030m,RelHum_015m,AbsHum_015m,WSpdMph_015m,PkWSpdMph_015m,PkWSpdMph_030m,VSSpdMph_015m,VSSpdMph_030m,BarPresMb_015m,Sigma_015m,Sigma_030m,SigPhi_015m,SigPhi_030m,WDir_015m,WDir_030m,PrecipIn_015m,event_E3_LowTemp_lt0,event_E4_HighWind_Peak_gt25,event_E5_LowWind_lt2,event_E6_HighTemp_gt24,WSpdMph_030m
0,TOWA,2017-01-01 05:00:00+00:00,3.6,3.6,97.4,6.0,2.2,5.9,10.2,0.0,-0.100000,985.3,26.5,15.5,18.9,12.6,268.0,274.0,0.015748,False,False,False,False,4.4
6,TOWA,2017-01-01 05:15:00+00:00,3.6,3.6,97.3,6.0,2.4,6.7,7.7,0.0,-0.100000,985.3,19.3,13.9,14.3,9.8,252.0,264.0,0.011811,False,False,False,False,4.7
12,TOWA,2017-01-01 05:30:00+00:00,3.6,3.7,97.4,6.0,2.8,6.0,6.8,-0.2,0.000064,985.2,25.3,15.0,13.6,11.0,255.0,260.0,0.011811,False,False,False,False,4.3
18,TOWA,2017-01-01 05:45:00+00:00,3.7,3.7,97.4,6.0,2.5,5.1,6.8,-0.2,-0.100000,985.0,29.4,16.6,14.4,8.3,268.0,263.0,0.007874,False,False,False,False,4.2
24,TOWA,2017-01-01 06:00:00+00:00,3.8,3.8,97.3,6.0,2.2,4.2,6.3,0.0,0.100000,985.1,25.0,13.1,14.9,10.8,268.0,274.0,0.007874,False,False,False,False,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261692,TOWA,2023-01-01 03:30:00+00:00,10.1,12.3,98.5,9.2,0.4,NaN,NaN,0.0,NaN,984.4,78.6,11.0,12.9,3.4,120.7,248.9,0.000000,False,False,False,False,2.6
1261698,TOWA,2023-01-01 03:45:00+00:00,10.2,11.8,98.6,9.3,0.2,NaN,NaN,0.0,NaN,984.4,66.5,15.0,5.7,3.8,273.6,263.7,0.000000,False,False,False,False,2.5
1261704,TOWA,2023-01-01 04:00:00+00:00,10.4,11.7,98.6,9.4,0.7,NaN,NaN,0.0,NaN,984.5,45.9,15.7,12.4,4.8,253.0,259.9,0.000000,False,False,False,False,3.1
1261710,TOWA,2023-01-01 04:15:00+00:00,10.2,11.7,98.6,9.3,0.4,NaN,NaN,0.0,NaN,984.5,91.1,12.3,11.0,4.2,294.1,268.0,0.000000,False,False,False,False,2.9


# VIZULIZATION!

In [10]:


from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ensure this exists
out_dir = Path("/home/jose/DATA_WEATHER_ORNL/viz_maps")
out_dir.mkdir(parents=True, exist_ok=True)

# Sanity check: filtered_dfs must be a dict of DataFrames keyed by tower name
print("Towers to visualize:", list(filtered_dfs.keys()))



def make_viz_map_for_tower(tower_df: pd.DataFrame, tower_name: str, out_dir: Path, null_display_top=40):
    """Create numeric × event heatmap for one tower and save output."""
    if tower_df is None or tower_df.empty:
        print(f"No data for {tower_name}")
        return

    df = tower_df.copy()
    # ensure timestamp
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    else:
        df.index = pd.to_datetime(df.index)
        df['timestamp'] = df.index

    # identify event columns and numeric sensor columns
    event_cols = [c for c in df.columns if str(c).startswith('event_')]
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # exclude boolean event columns that may be numeric
    numeric_cols = [c for c in numeric_cols if c not in event_cols]

    # numeric × event correlation (convert booleans to ints)
    if not numeric_cols or not event_cols:
        print(f"Skipping heatmap for {tower_name}: insufficient numeric or event columns")
        return

    combined = df[numeric_cols + event_cols].copy()
    for e in event_cols:
        # treat missing event flags as False
        combined[e] = combined[e].fillna(False).astype(int)

    corr_num_event = combined.corr().loc[numeric_cols, event_cols]

    if corr_num_event is None or corr_num_event.empty:
        print(f"No numeric × event correlations for {tower_name}")
        return

    sns.set(style="white")
    plt.figure(figsize=(8, max(4, 0.25 * len(numeric_cols))))
    sns.heatmap(corr_num_event.abs(), cmap='vlag', center=0, annot=False, cbar_kws={'label': '|Pearson r|'})
    plt.title(f"{tower_name} — numeric ↔ event |corr|")
    plt.tight_layout()
    out_path = out_dir / f"{tower_name}_numeric_event_heatmap.png"
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close()

    print(f"Saved heatmap for {tower_name}: {out_path}")


# Option B — all towers (recommended)
for tower_name, tower_df in tqdm(filtered_dfs.items(), desc="Making viz maps"):
    make_viz_map_for_tower(tower_df, tower_name, out_dir)


/home/jose/.conda/envs/ml_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Towers to visualize: ['TOWA', 'TOWB', 'TOWD', 'TOWF', 'TOWS', 'TOWY']


Making viz maps:  17%|█▋        | 1/6 [00:00<00:02,  2.00it/s]

Saved heatmap for TOWA: /home/jose/DATA_WEATHER_ORNL/viz_maps/TOWA_numeric_event_heatmap.png


Making viz maps:  33%|███▎      | 2/6 [00:00<00:01,  2.42it/s]

...

Saved heatmap for TOWD: /home/jose/DATA_WEATHER_ORNL/viz_maps/TOWD_numeric_event_heatmap.png


Making viz maps:  83%|████████▎ | 5/6 [00:01<00:00,  2.95it/s]

Saved heatmap for TOWF: /home/jose/DATA_WEATHER_ORNL/viz_maps/TOWF_numeric_event_heatmap.png
Saved heatmap for TOWS: /home/jose/DATA_WEATHER_ORNL/viz_maps/TOWS_numeric_event_heatmap.png


Making viz maps: 100%|██████████| 6/6 [00:02<00:00,  2.47it/s]

Saved heatmap for TOWY: /home/jose/DATA_WEATHER_ORNL/viz_maps/TOWY_numeric_event_heatmap.png
